In [6]:
import pandas as pd
import numpy as np 
import sys, os
import importlib.util

#need to get our product JSON from parent folder
p = os.path.abspath('../product_info_JSON.py')
spec = importlib.util.spec_from_file_location("product_info_JSON", p)
json_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(json_module)
#print(json_module.PRODUCT_INFORMATION)

# ---------------------------- End of Lookup Table work ---------------------------------

# Create the Abbott product table
Abbot_products_original = pd.read_csv("../../Data/Abbott/Abbott_scrape_data.csv",  usecols=json_module.PRODUCT_INFORMATION.keys()) #This could be replaced with output from other parser
Abbot_products = Abbot_products_original.copy() #to be used by future analysis
Abbot_products.drop_duplicates(keep='first', inplace=True)
len(Abbot_products)

309

In [85]:
Abbot_products.head()

,url,store,name,price,size_or_weight,availability,item_id,description,ingredients,allergin_info,...,Sizes,Form,Flavours,Case of X,Case of Y,clinical_indications,benefits,feature_table_rows,usage,entry_date
0,https://abbottstore.com/similac-pro-advance-in...,Abbott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22/05/2020 15:55:50
1,https://abbottstore.com/similac-pro-advance-in...,Abbott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22/05/2020 15:55:50
2,https://abbottstore.com/similac-pro-sensitive-...,Abbott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22/05/2020 15:55:50
3,https://abbottstore.com/advancepowder-30-8ozca...,Abbott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22/05/2020 15:55:50
4,https://abbottstore.com/similac-advance-infant...,Abbott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22/05/2020 15:55:50


In [86]:
#remove any dud values without name
Abbot_products = Abbot_products[~Abbot_products['name'].isna()]

In [88]:
#import each look up list - will be used to clean the data in next cell

import xml.etree.ElementTree as ET
#get all types and all measurements
root = ET.parse('../../Data/Abbott/Abbott_product_types_lookup.xml').getroot()
types_list = [i.text for i in root[0]]
types_list
measure_list = [i.text for i in root[1]]
measure_list
bulk_form_list = [i.text for i in root[2]]
#print(types_list)

In [89]:
#extract value from list that is found in another list
def exists_in_list(l, val):
    for i in val:
        if i in l:
            return i
    return None


a = Abbot_products['size_or_weight']
split_data = list()
#splits by '/'
for i in a:
    s = ' '.join(str(i).split())
    new_tuple = s.split('/')
    split_data.append(new_tuple)
    
 

In [90]:
    
lhs = list()
rhs = list()

#lhs to have weight metrics, type and weight number
#Rhs will have bulk sales availablility
for i in split_data:
    if len(i) == 2:
        rhs.append(i[1])
        lhs.append(i[0])
    else:
        lhs.append(i[0])
        rhs.append('')

wt_metric = list()
pr_type = list()
wt = list()
bulk_sales_type = list()
bulk_sales_nums = list()

#will create all the same length lists but with their specific values
for i in lhs:
    v = exists_in_list(i, measure_list)
    wt_metric.append(v)
    v = exists_in_list(i, types_list)
    pr_type.append(v)
    d = i.split(' ')
    prod_wt_num = [el for el in d if bool(re.search(r'\d', el))]
    
    if prod_wt_num:
        wt.append(prod_wt_num[0])
    else:
        wt.append('')


for i in rhs:
    v = exists_in_list(i, bulk_form_list)
    bulk_sales_type.append(v)
    d = i.split(' ')
    prod_wt_num = [el for el in d if bool(re.search(r'\d', el))]
    
    if prod_wt_num:
        bulk_sales_nums.append(prod_wt_num[0])
    else:
        bulk_sales_nums.append('')

    


In [91]:
Abbot_products.head()

,url,store,name,price,size_or_weight,availability,item_id,description,ingredients,allergin_info,...,Sizes,Form,Flavours,Case of X,Case of Y,clinical_indications,benefits,feature_table_rows,usage,entry_date
5,https://abbottstore.com/pro-sensitive-infant-f...,Abbott,Similac Pro-Sensitive Infant Formula Ready-to-...,57.96,2 fl oz bottle\n\t\t\t\t\t\t \n\t\t\t\t\t\t \n...,IN STOCK,67438,Similac Pro-Sensitive is the first infant form...,NaN,NaN,...,Case of 48,Ready-to-Feed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22/05/2020 15:55:50
10,https://abbottstore.com/similac-organic-ready-...,Abbott,Similac Organic Infant Formula Ready-to-Feed,51.99,2 fl oz bottle\n\t\t\t\t\t\t \n\t\t\t\t\t\t \n...,IN STOCK,56525,Similac Organic is designed to support your ba...,"Water, Organic Nonfat Milk, Organic Maltodextr...",Contains milk and soy ingredients.,...,Case of 48,Ready-to-Feed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22/05/2020 15:55:50
11,https://abbottstore.com/similac-soy-isomil-rea...,Abbott,Similac Soy Isomil Infant Formula Ready-to-Feed,51.99,2 fl oz bottle\n\t\t\t\t\t\t \n\t\t\t\t\t\t \n...,IN STOCK,56308,Soy-based formula for infants who need cow's m...,"Water (87%), Corn Syrup (6%), Soy Protein Isol...",Contains soy ingredients.,...,Case of 48,Ready-to-Feed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22/05/2020 15:55:50
13,https://abbottstore.com/go-and-grow-by-similac...,Abbott,Go & Grow by Similac Milk-Based Toddler Drink ...,63.50,2.25 lb can\n\t\t\t\t\t\t \n\t\t\t\t\t\t \n\t\...,IN STOCK,66838,TRUSTED FORMULA: Similac is the #1 brand chose...,NaN,NaN,...,Case of 3,Powder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22/05/2020 15:55:50
14,https://abbottstore.com/similac-for-spit-up-in...,Abbott,Similac For Spit-Up Infant Formula Ready-To-Feed,57.49,2 fl oz bottle\n\t\t\t\t\t\t \n\t\t\t\t\t\t \n...,IN STOCK,62139,Help your baby spit-up less frequently with ea...,"Water, Corn Syrup, Rice Starch, Milk Protein I...",Contains milk and soy ingredients.,...,Case of 48,Liquid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22/05/2020 15:55:50


In [92]:

can_numbers_1 = Abbot_products['Case of X'].str.extract('(\d+)')
can_numbers_2 = Abbot_products['Case of Y'].str.extract('(\d+)')
can_numbers = can_numbers_1 + ', ' + can_numbers_2
can_numbers = can_numbers.values.tolist()
#Abbot_products['product_type_counts'] = can_numbers
for i in range(0, len(can_numbers)):
    #print(can_numbers[i][0])
    if isinstance(can_numbers[i][0], str):
        bulk_sales_nums[i] = can_numbers[i][0]
        bulk_sales_type[i] = 'can'
        #print(can_numbers[i])


In [93]:
Abbot_products['product_format'] = pr_type
Abbot_products['product_measurement_metric'] = wt_metric
Abbot_products['product_measurement_numeric'] = wt
Abbot_products['bulk_sales_type'] = bulk_sales_type
Abbot_products['bulk_sales_numeric'] = bulk_sales_nums


In [94]:

Abbot_products.head()

,url,store,name,price,size_or_weight,availability,item_id,description,ingredients,allergin_info,...,clinical_indications,benefits,feature_table_rows,usage,entry_date,product_format,product_measurement_metric,product_measurement_numeric,bulk_sales_type,bulk_sales_numeric
5,https://abbottstore.com/pro-sensitive-infant-f...,Abbott,Similac Pro-Sensitive Infant Formula Ready-to-...,57.96,2 fl oz bottle\n\t\t\t\t\t\t \n\t\t\t\t\t\t \n...,IN STOCK,67438,Similac Pro-Sensitive is the first infant form...,NaN,NaN,...,NaN,NaN,NaN,NaN,22/05/2020 15:55:50,bottle,fl oz,2,Case,48
10,https://abbottstore.com/similac-organic-ready-...,Abbott,Similac Organic Infant Formula Ready-to-Feed,51.99,2 fl oz bottle\n\t\t\t\t\t\t \n\t\t\t\t\t\t \n...,IN STOCK,56525,Similac Organic is designed to support your ba...,"Water, Organic Nonfat Milk, Organic Maltodextr...",Contains milk and soy ingredients.,...,NaN,NaN,NaN,NaN,22/05/2020 15:55:50,bottle,fl oz,2,Case,48
11,https://abbottstore.com/similac-soy-isomil-rea...,Abbott,Similac Soy Isomil Infant Formula Ready-to-Feed,51.99,2 fl oz bottle\n\t\t\t\t\t\t \n\t\t\t\t\t\t \n...,IN STOCK,56308,Soy-based formula for infants who need cow's m...,"Water (87%), Corn Syrup (6%), Soy Protein Isol...",Contains soy ingredients.,...,NaN,NaN,NaN,NaN,22/05/2020 15:55:50,bottle,fl oz,2,Case,48
13,https://abbottstore.com/go-and-grow-by-similac...,Abbott,Go & Grow by Similac Milk-Based Toddler Drink ...,63.50,2.25 lb can\n\t\t\t\t\t\t \n\t\t\t\t\t\t \n\t\...,IN STOCK,66838,TRUSTED FORMULA: Similac is the #1 brand chose...,NaN,NaN,...,NaN,NaN,NaN,NaN,22/05/2020 15:55:50,can,lb,2.25,Case,3
14,https://abbottstore.com/similac-for-spit-up-in...,Abbott,Similac For Spit-Up Infant Formula Ready-To-Feed,57.49,2 fl oz bottle\n\t\t\t\t\t\t \n\t\t\t\t\t\t \n...,IN STOCK,62139,Help your baby spit-up less frequently with ea...,"Water, Corn Syrup, Rice Starch, Milk Protein I...",Contains milk and soy ingredients.,...,NaN,NaN,NaN,NaN,22/05/2020 15:55:50,bottle,fl oz,2,Case,48


In [95]:
# ------------- Setup dataframe and columns -----------------

del_cols = ['size_or_weight', 'benefits', 'clinical_indications', 'feature_table_rows', 'usage', 'Case of X', 'Case of Y',
           'Flavours', 'size_or_weight', 'availability', 'Sizes', 'footnotes', 'ingredients']

for c in del_cols:
    if c in Abbot_products.columns:
        del Abbot_products[c]
    

In [96]:
Abbot_products.head()

,url,store,name,price,item_id,description,allergin_info,serving_size_1,serving_size_2,serving_size_3,...,mineral_table_1,mineral_table_2,mineral_table_3,Form,entry_date,product_format,product_measurement_metric,product_measurement_numeric,bulk_sales_type,bulk_sales_numeric
5,https://abbottstore.com/pro-sensitive-infant-f...,Abbott,Similac Pro-Sensitive Infant Formula Ready-to-...,57.96,67438,Similac Pro-Sensitive is the first infant form...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Ready-to-Feed,22/05/2020 15:55:50,bottle,fl oz,2,Case,48
10,https://abbottstore.com/similac-organic-ready-...,Abbott,Similac Organic Infant Formula Ready-to-Feed,51.99,56525,Similac Organic is designed to support your ba...,Contains milk and soy ingredients.,Serving Size: 1000 mL (1.1 QT),NaN,NaN,...,NaN,NaN,NaN,Ready-to-Feed,22/05/2020 15:55:50,bottle,fl oz,2,Case,48
11,https://abbottstore.com/similac-soy-isomil-rea...,Abbott,Similac Soy Isomil Infant Formula Ready-to-Feed,51.99,56308,Soy-based formula for infants who need cow's m...,Contains soy ingredients.,Serving Size: 1000 mL (1.1 QT),NaN,NaN,...,NaN,NaN,NaN,Ready-to-Feed,22/05/2020 15:55:50,bottle,fl oz,2,Case,48
13,https://abbottstore.com/go-and-grow-by-similac...,Abbott,Go & Grow by Similac Milk-Based Toddler Drink ...,63.50,66838,TRUSTED FORMULA: Similac is the #1 brand chose...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Powder,22/05/2020 15:55:50,can,lb,2.25,Case,3
14,https://abbottstore.com/similac-for-spit-up-in...,Abbott,Similac For Spit-Up Infant Formula Ready-To-Feed,57.49,62139,Help your baby spit-up less frequently with ea...,Contains milk and soy ingredients.,Serving Size: 1000 mL (1.1 QT),NaN,NaN,...,NaN,NaN,NaN,Liquid,22/05/2020 15:55:50,bottle,fl oz,2,Case,48


In [97]:
Abbot_products['serving_size_1'] = Abbot_products['serving_size_1'].str.replace('Serving Size:', '')
Abbot_products['serving_size_1'] = Abbot_products['serving_size_1'].str.split('(', expand=True)[0]
Abbot_products['serving_size_1'] = Abbot_products['serving_size_1'].str.strip()
df = Abbot_products['serving_size_1'].str.split(' ', expand=True)

Abbot_products['serving_size_metric'] = df[1]
Abbot_products['serving_size_numeric'] = df[0]



In [98]:
del_cols = list()

for i in range(1, 4):
    del_cols.append('serving_size_' + str(i))
    del_cols.append('nutrient_table_' + str(i))
    del_cols.append('vitamin_table_' + str(i))
    del_cols.append('mineral_table_' + str(i))
    
for c in del_cols:
    if c in Abbot_products.columns:
        del Abbot_products[c]



In [99]:
Abbot_products.head()

,url,store,name,price,item_id,description,allergin_info,Form,entry_date,product_format,product_measurement_metric,product_measurement_numeric,bulk_sales_type,bulk_sales_numeric,serving_size_metric,serving_size_numeric
5,https://abbottstore.com/pro-sensitive-infant-f...,Abbott,Similac Pro-Sensitive Infant Formula Ready-to-...,57.96,67438,Similac Pro-Sensitive is the first infant form...,NaN,Ready-to-Feed,22/05/2020 15:55:50,bottle,fl oz,2,Case,48,NaN,NaN
10,https://abbottstore.com/similac-organic-ready-...,Abbott,Similac Organic Infant Formula Ready-to-Feed,51.99,56525,Similac Organic is designed to support your ba...,Contains milk and soy ingredients.,Ready-to-Feed,22/05/2020 15:55:50,bottle,fl oz,2,Case,48,mL,1000
11,https://abbottstore.com/similac-soy-isomil-rea...,Abbott,Similac Soy Isomil Infant Formula Ready-to-Feed,51.99,56308,Soy-based formula for infants who need cow's m...,Contains soy ingredients.,Ready-to-Feed,22/05/2020 15:55:50,bottle,fl oz,2,Case,48,mL,1000
13,https://abbottstore.com/go-and-grow-by-similac...,Abbott,Go & Grow by Similac Milk-Based Toddler Drink ...,63.50,66838,TRUSTED FORMULA: Similac is the #1 brand chose...,NaN,Powder,22/05/2020 15:55:50,can,lb,2.25,Case,3,NaN,NaN
14,https://abbottstore.com/similac-for-spit-up-in...,Abbott,Similac For Spit-Up Infant Formula Ready-To-Feed,57.49,62139,Help your baby spit-up less frequently with ea...,Contains milk and soy ingredients.,Liquid,22/05/2020 15:55:50,bottle,fl oz,2,Case,48,mL,1000


In [119]:

# Send to CSV
Abbot_products.to_csv("../../Data/Abbott/Abbot_products.csv")


In [123]:
#edit all tables, adding extra columns
import os.path

vitamins_df = pd.DataFrame(columns=['item_id','vitamin', 'measure', 'value', 'perc_daily_recom'])
for prod in Abbot_products['item_id'].tolist():
    p = "../../Data/Abbott/Vitamin_tables/vitamin_table_" + prod + ".csv"
    #print(p)
    if os.path.exists(p):
        #print('----')
        vit_table = pd.read_csv(p)
        
        try:
            df = pd.DataFrame(columns=['item_id','vitamin', 'measure', 'value', 'perc_daily_recom'])
            df[['vit', 'measure']] = vit_table['Unnamed: 0'].str.split(',', expand=True)
            df[['measure', 'value']] = df['measure'].str.split(':', expand=True)
            df['item_id'] = prod 
            df['perc_daily_recom'] = vit_table['% Daily Value*']
            vitamins_df = vitamins_df.append(df)
            
        except:
            print('Could not split measures')
            #print(vit_table)
            #break
            
    
vitamins_df.to_csv("../../Data/Abbott/Abbot_vitamins_full.csv", index=False)
        

Could not split measures
Could not split measures
Could not split measures
Could not split measures
Could not split measures
Could not split measures
Could not split measures
Could not split measures
Could not split measures
Could not split measures
Could not split measures
Could not split measures


In [121]:
#edit all tables, adding extra columns
import os.path

nutrients_df = pd.DataFrame(columns=['item_id','nutrient', 'measure', 'value', 'perc_daily_recom'])
for prod in Abbot_products['item_id'].tolist():
    p = "../../Data/Abbott/Nutrition_tables/nutrient_table_" + prod + ".csv"
    #print(p)
    if os.path.exists(p):
        #print('----')
        nutr_table = pd.read_csv(p)
        
        try:
            df = pd.DataFrame(columns=['item_id','nutrient', 'measure', 'value', 'perc_daily_recom'])
            df[['nutrient', 'measure']] = nutr_table['Unnamed: 0'].str.split(',', expand=True)
            df[['measure', 'value']] = df['measure'].str.split(':', expand=True)
            df['item_id'] = prod 
            df['perc_daily_recom'] = nutr_table['% Daily Value*']
            nutrients_df = nutrients_df.append(df)
            
        except:
            print('Could not split measures')
            #print(vit_table)
            #break
            
    
nutrients_df.to_csv("../../Data/Abbott/Abbot_nutrients_full.csv", index=False)
        

In [122]:
#edit all tables, adding extra columns
import os.path

minerals_df = pd.DataFrame(columns=['item_id','mineral', 'measure', 'value', 'perc_daily_recom'])
for prod in Abbot_products['item_id'].tolist():
    p = "../../Data/Abbott/Mineral_tables/mineral_table_" + prod + ".csv"
    #print(p)
    if os.path.exists(p):
        #print('----')
        min_table = pd.read_csv(p)
        
        try:
            df = pd.DataFrame(columns=['item_id','mineral', 'measure', 'value', 'perc_daily_recom'])
            df[['mineral', 'measure']] = min_table['Unnamed: 0'].str.split(',', expand=True)
            df[['measure', 'value']] = df['measure'].str.split(':', expand=True)
            df['item_id'] = prod 
            df['perc_daily_recom'] = min_table['% Daily Value*']
            minerals_df = minerals_df.append(df)
            
        except:
            print('Could not split measures')
            #print(vit_table)
            #break
            
    
minerals_df.to_csv("../../Data/Abbott/Abbot_mineral_full.csv", index=False)
        

Could not split measures
Could not split measures
Could not split measures
Could not split measures
Could not split measures
Could not split measures
Could not split measures
Could not split measures
Could not split measures
Could not split measures


In [13]:
# -------------------------------------------

# Set up table which shows all the ingredients for products vertically for analytics
Abbot_products_ingredients_item_type = Abbot_products_original

# Changed from Name to SKU ID "item_type"
Abbot_products_ingredients_item_type = Abbot_products_ingredients_item_type[pd.notnull(Abbot_products_ingredients_item_type['item_id'])]
Abbot_products_ingredients = Abbot_products_ingredients_item_type[['item_id','ingredients']]
Abbot_products_name = Abbot_products_ingredients_item_type['item_id']
Abbot_main_ingredients = pd.concat([Abbot_products_name,
                                    Abbot_products_ingredients['ingredients'].str.split(', ', expand=True)], axis=1)
Abbot_main_ingredients = pd.melt(Abbot_main_ingredients, id_vars = ["item_id"])
Abbot_main_ingredients.dropna(inplace=True)
Abbot_main_ingredients = Abbot_main_ingredients[(Abbot_main_ingredients['value'] != 0)]

Abbot_main_ingredients.rename(columns={'value' : 'ingredient'},inplace=True)
del Abbot_main_ingredients['variable']
Abbot_main_ingredients.head()
Abbot_main_ingredients.sort_values('item_id', inplace=True, ascending=True)
Abbot_main_ingredients.drop_duplicates(keep='first', inplace=True)
Abbot_main_ingredients


# Remove white space and "AND" 
Abbot_main_ingredients.ingredient = Abbot_main_ingredients.ingredient.str.strip()
Abbot_main_ingredients['ingredient'] = Abbot_main_ingredients['ingredient'].str.replace("and ", "", regex=False)
Abbot_main_ingredients['ingredient'] = Abbot_main_ingredients['ingredient'].str.replace(".", "", regex=False)

# Send to CSV
Abbot_main_ingredients.to_csv("../Data/Abbott/ingredients.csv")

In [9]:

# -----------

# Set up table which shows all the flavours for products vertically for analytics
# Changed name to SKU "item_type"

Abbot_product_flavours_item_type = Abbot_products_original
 #This could be replaced with output from other parser
    
Abbot_product_flavours_item_type = Abbot_product_flavours_item_type[pd.notnull(Abbot_product_flavours_item_type['item_id'])]
Abbot_products_name = Abbot_product_flavours_item_type['item_id']
Abbot_product_flavours = Abbot_product_flavours_item_type[['item_id','Flavours']]
Abbot_product_flavours = pd.concat([Abbot_products_name,
                                    Abbot_product_flavours['Flavours'].str.split(', ', expand=True)], axis=1)


Abbot_product_flavours = pd.melt(Abbot_product_flavours, id_vars = ["item_id"])
Abbot_product_flavours['value'] = Abbot_product_flavours['value'].astype(str)

Abbot_product_flavours['value'] = Abbot_product_flavours['value'].str.replace("]", "", regex=False)
Abbot_product_flavours['value'] = Abbot_product_flavours['value'].str.replace("[", "", regex = False)
Abbot_product_flavours['value'] = Abbot_product_flavours['value'].str.replace("'", "", regex=False)
Abbot_product_flavours.dropna(inplace=True)
Abbot_product_flavours = Abbot_product_flavours[(Abbot_product_flavours['value'] != 0)]
del Abbot_product_flavours['variable']
Abbot_product_flavours.rename(columns={'value' : 'flavour'},inplace=True)

Abbot_product_flavours.sort_values('item_id', inplace=True, ascending=True)
Abbot_product_flavours.drop_duplicates(keep='first', inplace=True)

Abbot_product_flavours.to_csv("../../Data/Abbott/flavours.csv", index=False)
